# Bayesian Modeling for Political Analysis

## Steps

1. Imports: Packages to be used by the Bayesian Modeling Process
2. Constants: Variables that are used throughout the modeling process
3. Fit Bayesian Models to the data
    - 
    - 
    - 
    - 
    - 
4. 

In [1]:
import pandas as pd
import sqlite3 as db
import numpy as np
import pymc3 as pm
import theano
floatX = theano.config.floatX
import matplotlib.pyplot as plt
import theano.tensor as T
import pickle

## Constants

## Simple Bayesian Model Estimating One Parameter

In [ ]:
with pm.Model() as model:
    # Priors
    theta = pm.Beta('theta', alpha = 0.1, beta = 0.1, shape = regions)
    y = pm.Bernoulli('y', p = theta[idx], observed = data)

## Data Transformation and Preparation

In [ ]:
for col in DEMO_COLS:
    df.loc[:,col] = df[col].apply(lambda x: x if x >= 0 else 0)

In [ ]:
X = df[['REPUBLICAN_PCT','YR','RGN_DESC'] + DEMO_COLS + STD_COLS  + SCALE_COLS]# + NN_COLS]
X.loc[:,'MED_HH_INC'] = (X['MED_HH_INC'] / 1000).astype(int)
X.loc[:,'PER_CAP_INC'] = (X['PER_CAP_INC'] / 1000).astype(int)
X.loc[:,'POP_EMPLOYED'] = X['POP_EMPLOYED'] / X['TOT_POP_CNTY'] * 100
X.loc[:,'REPUBLICAN_PCT'] = X['REPUBLICAN_PCT'] * 100
X.loc[:,'TOT_POP_CNTY'] = np.log(X['TOT_POP_CNTY'] / 1000) 

X.head()

In [ ]:
_ = plt.hist(X['REPUBLICAN_PCT'], bins='auto')
plt.title("Histograms of REPUBLICAN_PCT")
plt.show()

In [ ]:
X16 = X[X['YR'] == 2016]
X16.drop(columns = ['YR'], inplace = True)
X12 = X[X['YR'] == 2012]

regions = X16.RGN_DESC.unique()
regions = pd.DataFrame(regions, columns=['REGION'])
regions['i'] = regions.index

X16 = pd.merge(X16, regions, left_on='RGN_DESC', right_on='REGION', how='left')
X16 = X16.rename(columns = {'i': 'ix_region'}).drop('REGION', 1)

X_train, X_test, y_train, y_test = train_test_split(X16.iloc[:,2:], X16['REPUBLICAN_PCT'], test_size = 0.3, random_state=42)

region = X_train.ix_region.values
X_train.drop(columns = 'ix_region', inplace = True)
#you are modelling the standard deviation of a normal distribution with a normal. The test point for that is 0.0, which has 0 probability of occurring.

## Partial Pooling Method

In [ ]:
with pm.Model() as model:
    # Priors
    intercept = pm.Normal('intercept', mu= 50, sigma=10, shape = len(regions['i']))
    grp_sd = pm.Uniform('grp_sd', 0, 10)

    y_hat = intercept[region] 
    for ix, var in enumerate(X_train):
        y_hat += pm.Normal("beta_{}".format(ix), mu= 0, sigma = grp_sd, shape=len(regions['i']))[region] * X_train[col].values

    # Model error
    sigma_y = pm.HalfCauchy('sigma_y', beta = 1)
    
    # Creating the model requires a formula and data (and optionally a family)
    likelihood = pm.Normal('y', mu=y_hat, sd=sigma_y, observed=y_train.values)

    # Perform Markov Chain Monte Carlo sampling letting PyMC3 choose the algorithm
    trace = pm.sample(2000, step = pm.NUTS(), cores = 1)#, init = 'advi+adapt_diag_grad'

    model_formula = 'REPUBLICAN_PCT = '
    for variable in trace.varnames:
        model_formula += ' %0.2f * %s +' % (np.mean(trace[variable]), variable)
        
    print(model_formula)

In [ ]:
ax = pm.forestplot(trace, var_names=['intercept'])
ax[0].set_yticklabels(regions['REGION'].values)
ax[0].set_title('Regions Intercept');

In [ ]:
ax = pm.forestplot(trace, var_names=['beta_34'])
ax[0].set_yticklabels(regions['REGION'].values)
ax[0].set_title('Total Population by Region');

In [ ]:
ax = pm.traceplot(trace, var_names=['intercept', 'beta_33']);

## Mixed Models

In [ ]:
RGN_TRACE = {}

for rgn in X['RGN_DESC'].unique():
    X_RGN = X[X['RGN_DESC'] == rgn]
    df12 = X_RGN[X_RGN['YR'] == 2012]
    df12.drop(columns = ['RGN_DESC','YR'], inplace = True)
    
    df16 = X_RGN[X_RGN['YR'] == 2016]
    df16.drop(columns = ['RGN_DESC','YR'], inplace = True)
    
    X_train, X_test, y_train, y_test = train_test_split(df16, df16['REPUBLICAN_PCT'], test_size = 0.1, random_state=42)
    
    formula = "REPUBLICAN_PCT ~ " + " + ".join(list(df12)[1:])
    with pm.Model() as model:
        #set variable means and sd
        grp_mean = pm.Normal('grp_mean', mu= 50, sigma=10)
        grp_sd = pm.Uniform('grp_sd', 0, 10)
        
        # The prior for the data likelihood is a Normal REPUBLICAN_PCT
        priors = {'Intercept': pm.Normal.dist(mu=df12['REPUBLICAN_PCT'].mean(), 
                                            sigma=df12['REPUBLICAN_PCT'].std())}
        for col in list(df12)[1:]:
            priors[col] =  pm.Normal.dist(mu=grp_mean, sigma=grp_sd)

        # Creating the model requires a formula and data (and optionally a family)
        pm.GLM.from_formula(formula, data = X_train, family = pm.glm.families.Normal(), priors = priors)

        # Perform Markov Chain Monte Carlo sampling letting PyMC3 choose the algorithm
        trace = pm.sample(2000, step = pm.NUTS(), cores = 1)

    RGN_TRACE[rgn] = trace
    
    evaluate_trace(trace, X_train, X_test, y_train, y_test)

In [ ]:
#get a table to print out all of the coefficients together
i = 0
full_df = []
for key, normal_trace in RGN_TRACE.items():
    if i == 0:
        column_names = ['Region'] + pm.summary(normal_trace).index.tolist()
        i+=1
    row = [key] + pm.summary(normal_trace)['mean'].round(2).values.tolist()
    full_df.append(row)
    
df_coef = pd.DataFrame(full_df, columns = column_names)
df_coef

In [ ]:
DEMO_PATH = PROJ + 'data/raw/demographics/'
IND_PATH = PROJ + 'data/raw/indicators/'
PROC_PATH = PROJ + 'data/processed/'
SQL_US_ANALYSIS =  db.connect(PROC_PATH + 'us_pop_factors.db')
DEMO_EX = [ 'IA_FEMALE_BOOMER', 'IA_FEMALE_OLD', 'IA_FEMALE_YOUNG', 'IA_FEMALE_YOUNG_PROF', 'IA_MALE_BOOMER', 'IA_MALE_OLD', 
              'IA_MALE_YOUNG', 'IA_MALE_YOUNG_PROF', 'NA_FEMALE_BOOMER', 'NA_FEMALE_OLD', 'NA_FEMALE_YOUNG', 
              'NA_FEMALE_YOUNG_PROF', 'NA_MALE_BOOMER', 'NA_MALE_OLD', 'NA_MALE_YOUNG', 'NA_MALE_YOUNG_PROF', 
              'TOM_FEMALE_BOOMER', 'TOM_FEMALE_OLD', 'TOM_FEMALE_YOUNG', 'TOM_FEMALE_YOUNG_PROF', 
              'TOM_MALE_BOOMER', 'TOM_MALE_OLD', 'TOM_MALE_YOUNG', 'TOM_MALE_YOUNG_PROF']
DEMO_COLS = ['AA_FEMALE_BOOMER', 'AA_FEMALE_OLD', 'AA_FEMALE_YOUNG', 'AA_FEMALE_YOUNG_PROF', 'AA_MALE_BOOMER', 
              'AA_MALE_OLD', 'AA_MALE_YOUNG', 'AA_MALE_YOUNG_PROF', 'BA_FEMALE_BOOMER', 'BA_FEMALE_OLD', 
              'BA_FEMALE_YOUNG', 'BA_FEMALE_YOUNG_PROF', 'BA_MALE_BOOMER', 'BA_MALE_OLD', 'BA_MALE_YOUNG', 
              'BA_MALE_YOUNG_PROF', 'H_FEMALE_BOOMER', 'H_FEMALE_OLD', 'H_FEMALE_YOUNG', 'H_FEMALE_YOUNG_PROF', 
              'H_MALE_BOOMER', 'H_MALE_OLD', 'H_MALE_YOUNG', 'H_MALE_YOUNG_PROF',
              'WA_FEMALE_BOOMER', 'WA_FEMALE_OLD', 'WA_FEMALE_YOUNG', 'WA_FEMALE_YOUNG_PROF', 
              'WA_MALE_BOOMER', 'WA_MALE_OLD', 'WA_MALE_YOUNG', 'WA_MALE_YOUNG_PROF']
STD_COLS = ['MED_HH_INC', 'PER_CAP_INC','TOT_POP_CNTY']
NN_COLS = ['URBAN','NEAR_METRO','METRO','ADJACENT']
SCALE_COLS = ['PCT_LESS_HS', 'PCT_HS', 'PCT_SOME_BA', 'PCT_EQ_MORE_BA', 'PCT_POV_U18', 
              'PCT_POV_ALL', 'PCT_DEEP_POV_ALL', 'PCT_DEEP_POV_U18', 'PCT_CHG_EMPLOY_0710', 'PCT_CHG_EMPLOY_0718', 
              'PCT_CHG_EMPLOY_1018', 'PCT_CHG_EMPLOY_1718', 'AGRICULTURE', 'MINING', 'CONSTRUCTION',
              'MANUFACTURING', 'TRADE', 'TRANSPORTATION', 'INFORMATION', 'FIRE', 'SERVICES', 'GOVERNMENT', 'POP_EMPLOYED']
REPORT_PATH = PROJ + 'reports/'
COLS_USE = DEMO_COLS + STD_COLS + SCALE_COLS
enc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# Evalute the MCMC trace and compare to ml models
def evaluate_trace(trace, X_train, X_test, y_train, y_test):
    #fit the linear regression to the dataset and get the prediction
    regressor = LinearRegression()  
    regressor.fit(X_train[X_train.columns.difference(['REPUBLICAN_PCT'])], y_train)
    y_pred = regressor.predict(X_test[X_test.columns.difference(['REPUBLICAN_PCT'])])
    
    # Dictionary of all sampled values for each parameter
    var_dict = {}
    for variable in trace.varnames:
        var_dict[variable] = trace[variable]
        
    # Results into a dataframe
    var_weights = pd.DataFrame(var_dict)
    
    # Means for all the weights
    var_means = var_weights.mean(axis=0)
    
    # Create an intercept column
    X_test['Intercept'] = 1
    
    # Align names of the test observations and means
    names = X_test.columns[1:]
    X_test = X_test.loc[:, names]
    var_means = var_means[names]
    
    # Calculate estimate for each test observation using the average weights
    results = pd.DataFrame(index = X_test.index, columns = ['estimate'])

    for row in X_test.iterrows():
        results.loc[row[0], 'estimate'] = np.dot(np.array(var_means), np.array(row[1]))

    actual = np.array(y_test)
    b_errors = results['estimate'] - actual
    n_errors = y_pred - actual
    
    print('Bayesian LR RMSE: %0.2f' % np.sqrt(np.mean(b_errors ** 2)))
    print('Normal LR RMSE: %0.2f' % np.sqrt(np.mean(n_errors ** 2)))